We insert the segmented (cropped) volume (with kmeans) inside a 256x256x256 volumes of zero. We use it to differentiate active signal/background region. Then we calculate the metrics.

In [7]:
import numpy as np

binary_mask_path = '../data/battery/4_battery_clean_crops/gt_data/2means_160x160x256.npy'
binary_mask = np.load(binary_mask_path)
print(f'binary mask loaded with shape {binary_mask.shape}')

binary mask loaded with shape (160, 160, 256)


In [3]:
np.unique(binary_mask)

array([False,  True])

In [4]:
depth, height, width = binary_mask.shape
center = (128, 128, 128)

In [5]:
gt_volume = np.zeros(shape=(256, 256, 256), dtype=np.bool_)
gt_volume[
    center[0] - (depth//2) : center[0] + (depth//2),
    center[1] - (height//2) : center[1] + (height//2),
    :,
] = binary_mask

np.unique(gt_volume)

array([False,  True])

In [6]:
np.save('../data/z-bin/2means_256x256x256.npy', gt_volume)
print('saved.')

saved.


In [24]:
gt_volume.shape

(256, 256, 256)

Implement now metrics such as SNR and CNR.

In [15]:
def compute_snr_cnr(test_volume, gt_volume):
    """
    Compute SNR and CNR for a given test volume and binary ground-truth volume.

    Parameters:
        test_volume (numpy.ndarray): 3D numpy array representing the test volume.
        gt_volume (numpy.ndarray): 3D binary numpy array (1 for foreground, 0 for background).

    Returns:
        snr (float): Signal-to-Noise Ratio.
        cnr (float): Contrast-to-Noise Ratio.
    """

    # Check dimensions
    if gt_volume.shape != test_volume.shape:
        raise Exception('Error: test volume and ground truth must have same dimensions.')

    # Extract the foreground and background regions based on gt_volume
    foreground = test_volume[gt_volume == True]
    background = test_volume[gt_volume == False]

    # Compute the mean of the foreground (signal)
    mu_i = np.mean(foreground)

    # Compute the mean and standard deviation of the background (noise)
    mu_b = np.mean(background)
    sigma_b = np.std(background)

    # Compute SNR (Signal-to-Noise Ratio)
    if sigma_b == 0:
        snr = np.inf  # Handle division by zero case
    else:
        snr = 20 * np.log10(mu_i / sigma_b)

    # Compute CNR (Contrast-to-Noise Ratio)
    if sigma_b == 0:
        cnr = np.inf  # Handle division by zero case
    else:
        cnr = 20 * np.log10(np.abs(mu_i - mu_b) / sigma_b)

    return snr, cnr

Experiment 1

In [16]:
experiment_dir = '../data/battery/4_battery_clean_crops/'

In [21]:
exp_1_names = [
    'exp_1/saxnerf/volume_180.npy',
    'exp_1/saxnerf/volume_90.npy',
    'exp_1/saxnerf/volume_45.npy',
    'exp_1/r2gaus/volume_180.npy',
    'exp_1/r2gaus/volume_90.npy',
    'exp_1/r2gaus/volume_45.npy'
]

exp_2_names = [
    'exp_2/saxnerf/volume_30_150.npy',
    'exp_2/saxnerf/volume_45_135.npy',
    'exp_2/saxnerf/volume_60_120.npy',
    'exp_2/r2gaus/volume_30_150.npy',
    'exp_2/r2gaus/volume_45_135.npy',
    'exp_2/r2gaus/volume_60_120.npy',
]

# Set here the volumes you want to test
volume_names = exp_2_names

test_volumes = []
for volume_name in volume_names:
    volume = np.load(experiment_dir + volume_name)
    test_volumes.append(volume)

print(f'{len(test_volumes)} volumes loaded.')

6 volumes loaded.


In [22]:
for id in range(len(volume_names)):
    snr, cnr = compute_snr_cnr(test_volumes[id], gt_volume)
    print(f'+------- {volume_names[id]} -------+')
    print(f"SNR: {snr:.2f} dB")
    print(f"CNR: {cnr:.2f} dB")
    print()

+------- exp_2/saxnerf/volume_30_150.npy -------+
SNR: 16.59 dB
CNR: 10.57 dB

+------- exp_2/saxnerf/volume_45_135.npy -------+
SNR: 16.44 dB
CNR: 9.69 dB

+------- exp_2/saxnerf/volume_60_120.npy -------+
SNR: 13.94 dB
CNR: 7.04 dB

+------- exp_2/r2gaus/volume_30_150.npy -------+
SNR: 9.04 dB
CNR: 1.68 dB

+------- exp_2/r2gaus/volume_45_135.npy -------+
SNR: 10.75 dB
CNR: 3.77 dB

+------- exp_2/r2gaus/volume_60_120.npy -------+
SNR: 8.42 dB
CNR: -0.82 dB



These are custom video-style metrics but we are lacking reference papers. 

In [1]:
import numpy as np
from skimage.metrics import structural_similarity as ssim

def frame_to_frame_difference(volume):
    """Compute the mean absolute difference between consecutive frames."""
    diffs = [np.abs(volume[i] - volume[i + 1]).mean() for i in range(len(volume) - 1)]
    return np.mean(diffs)

def frame_to_frame_mse(volume):
    """Compute the mean squared error between consecutive frames."""
    mses = [np.mean((volume[i] - volume[i + 1])**2) for i in range(len(volume) - 1)]
    return np.mean(mses)

def frame_to_frame_ssim(volume):
    """Compute the average SSIM between consecutive frames."""
    ssims = [
        ssim(volume[i], volume[i + 1], data_range=volume[i + 1].max() - volume[i + 1].min())
        for i in range(len(volume) - 1)
    ]
    return np.mean(ssims)

def total_variation_time(volume):
    """Compute the total variation across consecutive frames."""
    tv = np.sum([np.abs(volume[i] - volume[i + 1]).sum() for i in range(len(volume) - 1)])
    return tv


In [3]:
# Example usage with a 3D numpy volume
path = '/home/dlboxadmin/Desktop/code-box/exploration/data/battery/4_battery_clean_crops/exp_3/r2gaus/50_random_projs.npy'

volume = np.load(path)  # Replace this with your actual reconstructed volume

print("Frame-to-Frame Difference:", frame_to_frame_difference(volume))
print("Frame-to-Frame MSE:", frame_to_frame_mse(volume))
print("Frame-to-Frame SSIM:", frame_to_frame_ssim(volume))
print("Total Variation in Time:", total_variation_time(volume))


Frame-to-Frame Difference: 0.009967527
Frame-to-Frame MSE: 0.0003783156
Frame-to-Frame SSIM: 0.936653716523297
Total Variation in Time: 166574.12
